# Getting Started

In [ ]:
# Naming conventions (to be modified)
# 1. Use short variable names but not too short. So that any variable name can
# be uniquely found using Ctrl+F
# 2. Use number or underscore to concatenate words in one variable name

## Reset notebook

In [155]:
%reset -f
%whos

Interactive namespace is empty.


## Predefine variables

In [160]:
%reset -f
import matplotlib.pyplot as plt
import matplotlib.tri as tri
plt.style.use('seaborn-poster')
#plt.style.use('ggplot')

import copy,os,subprocess,time
import numpy as np
import numpy.matlib
from scipy import sparse
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import spsolve

#set physical constants
echarge=1.602e-19 #elementary charge [C]
perm0=8.85e-12 #vacuum permittivity [F/m]
kA=6.022e23 #Avogadro's constant [1/mol]
kB=1.381e-23 #Boltzmann's constant [J/K]
kF=96485.0 #Faraday's constant [As/mol] equal to kA*echarge

#predefine user input variables
mesh_prefix='placeholder' #mesh prefix

perm_a=0.0 #relative permittivity of electrolyte [SI]
perm_i=0.0 #relative permittivity of solid [SI]
temperature=0.0 #ambient temperature [K]

n_ion=2 #number of ion types (0 means no diffuse layer)
c_ion=[0.0]*n_ion #ion concentration at infinity [mol/m^3]
z_ion=[0.0]*n_ion #ion valence or charge number [SI]
q_ion=[0.0]*n_ion #electric charge (q) in [Coulombs]
debye_len=[0.0]*n_ion #Debye length [m]
mu_a=[0.0]*n_ion #ion mobility in electrolyte [m^2/(Vs)]
mu_i=[0.0]*n_ion #ion mobility in solid [m^2/(Vs)]
Diff_a=[0.0]*n_ion #diffusion coefficient in electrolyte [m^2/s]
Diff_i=[0.0]*n_ion #diffusion coefficient in solid [m^2/s]

#predefine mesh variables
ndim=2 #2D simulation
nnode=0 #number of total nodes
nelem=0 #number of total triangular elements
nedge=0 #number of total line segments

nodes=np.zeros((nnode,ndim),dtype=float) #[X,Y]
elements=np.zeros((nelem,ndim+1),dtype=int) #[ind_a,ind_b,ind_c]
edges=np.zeros((nedge,ndim),dtype=int) #[ind_a,ind_b]

node_flags=np.zeros(nnode,dtype=bool) #node flags
elem_flags=np.zeros(nelem,dtype=bool) #zones
edge_flags=np.zeros(nedge,dtype=bool) #edge flags

#predefine mesh indexing variables
in_air=np.zeros(nelem,dtype=bool)
in_electrolyte=np.zeros(nelem,dtype=bool)
in_solid=np.zeros(nelem,dtype=bool)
in_active_zones=np.zeros(nelem,dtype=bool)
in_stern=np.zeros(nedge,dtype=bool)

with_first_kind_bc=np.zeros((nedge,n_ion+1),dtype=bool)
with_third_kind_bc=np.zeros((nedge,n_ion+1),dtype=bool)

on_first_kind_bc=np.zeros((nnode,n_ion+1),dtype=bool)
on_third_kind_bc=np.zeros((nnode,n_ion+1),dtype=bool)

#predefine material properties for air, electrolyte, and solid
c_xe=np.zeros((nelem,n_ion+1,n_ion+1),dtype=float)
c_ye=np.zeros((nelem,n_ion+1,n_ion+1),dtype=float)

alpha_xe=np.zeros((nelem,n_ion+1,n_ion+1),dtype=float)
alpha_ye=np.zeros((nelem,n_ion+1,n_ion+1),dtype=float)

beta_xe=np.zeros((nelem,n_ion+1,n_ion+1),dtype=float)
beta_ze=np.zeros((nelem,n_ion+1,n_ion+1),dtype=float)

gamma_xe=np.zeros((nelem,n_ion+1),dtype=float)
gamma_ye=np.zeros((nelem,n_ion+1),dtype=float)

a_e=np.zeros((nelem,n_ion+1,n_ion+1),dtype=float)
f_e=np.zeros((nelem,n_ion+1),dtype=float)
g_s0=np.zeros((nedge,n_ion+1),dtype=float)
q_s0=np.zeros((nedge,n_ion+1),dtype=float)
s_n0=np.zeros((nnode,n_ion+1),dtype=float)
a_n0=np.zeros((nnode,n_ion+1,n_ion+1),dtype=float)
f_n0=np.zeros((nnode,n_ion+1),dtype=float)

#predefine material properties for stern layer
c_xs=np.zeros((nedge,n_ion+1,n_ion+1),dtype=float)
alpha_xs=np.zeros((nedge,n_ion+1,n_ion+1),dtype=float)
beta_xs=np.zeros((nedge,n_ion+1,n_ion+1),dtype=float)
gamma_xs=np.zeros((nedge,n_ion+1),dtype=float)
a_s=np.zeros((nedge,n_ion+1,n_ion+1),dtype=float)
f_s=np.zeros((nedge,n_ion+1),dtype=float)
g_s1=np.zeros((nedge,n_ion+1),dtype=float)
q_s1=np.zeros((nedge,n_ion+1),dtype=float)
s_n1=np.zeros((nnode,n_ion+1),dtype=float)
a_n1=np.zeros((nnode,n_ion+1,n_ion+1),dtype=float)
f_n1=np.zeros((nnode,n_ion+1),dtype=float)


#https://www.wrighters.io/how-to-view-all-your-variables-in-a-jupyter-notebook/
%whos ndarray


Variable             Type       Data/Info
-----------------------------------------
a_e                  ndarray    0x3x3: 0 elems, type `float64`, 0 bytes
a_n0                 ndarray    0x3x3: 0 elems, type `float64`, 0 bytes
a_n1                 ndarray    0x3x3: 0 elems, type `float64`, 0 bytes
a_s                  ndarray    0x3x3: 0 elems, type `float64`, 0 bytes
alpha_xe             ndarray    0x3x3: 0 elems, type `float64`, 0 bytes
alpha_xs             ndarray    0x3x3: 0 elems, type `float64`, 0 bytes
alpha_ye             ndarray    0x3x3: 0 elems, type `float64`, 0 bytes
beta_xe              ndarray    0x3x3: 0 elems, type `float64`, 0 bytes
beta_xs              ndarray    0x3x3: 0 elems, type `float64`, 0 bytes
beta_ze              ndarray    0x3x3: 0 elems, type `float64`, 0 bytes
c_xe                 ndarray    0x3x3: 0 elems, type `float64`, 0 bytes
c_xs                 ndarray    0x3x3: 0 elems, type `float64`, 0 bytes
c_ye                 ndarray    0x3x3: 0 elems, type

In [141]:
print(nodes)

[0. 2.]


# Global variables

## User inputs

In [75]:
#Modify user inputs below
mesh_prefix='sphere_2d'

rel_perma=80.0 #relative permittivity of electrolyte
rel_permi=4.5 #relative permittivity of solid

temperature=293.0 #ambient temperature in [Kelvin]
c_ion=[1.0,1.0] #ion concentration at infinity [mol/m^3]
z_ion=[1,1] #ion valence or charge number [SI]

mu_a=[5e-8,5e-8] #mobility of sodium cation in bulk electrolyte [m^2/(Vs)]
mu_s=[5e-11,5e-11] #mobility in Stern layer [m^2/(Vs)]
    
#https://www.wrighters.io/how-to-view-all-your-variables-in-a-jupyter-notebook/
%whos int
print('')
%whos


Variable      Type     Data/Info
--------------------------------
c_ion         list     n=2
echarge       float    1.602e-19
kA            float    6.022e+23
kB            float    1.381e-23
kF            float    96485.0
mesh_prefix   str      sphere_2d
mu_a          list     n=2
mu_s          list     n=2
perm0         float    8.85e-12
rel_perma     float    80.0
rel_permi     float    4.5
temperature   float    293.0
z_ion         list     n=2


## Preallocated variables

In [ ]:
if __name__=='__main__':
    #global variables
    nnode=0 #number of nodes
    nelem=0 #number of triangular elements
    nedge=0 #number of line segments
    
    #physicsl constants
    
    
    
    print('Done')

## List of global variables

In [112]:
#https://www.wrighters.io/how-to-view-all-your-variables-in-a-jupyter-notebook/
# print('All variables')
# print('*' * 80)  # Drawings Horizontal Divider Line
# %whos
# print('')
print('<Variables sorted by Type>')
print('*' * 80)  # Drawings Horizontal Divider Line
%whos module
print('')
%whos function
print('')
%whos type 

<Variables sorted by Type>
********************************************************************************
Variable     Type      Data/Info
--------------------------------
copy         module    <module 'copy' from '/hom<...>/lib/python3.10/copy.py'>
np           module    <module 'numpy' from '/ho<...>kages/numpy/__init__.py'>
numpy        module    <module 'numpy' from '/ho<...>kages/numpy/__init__.py'>
os           module    <module 'os' from '/home/<...>er/lib/python3.10/os.py'>
plt          module    <module 'matplotlib.pyplo<...>es/matplotlib/pyplot.py'>
sparse       module    <module 'scipy.sparse' fr<...>cipy/sparse/__init__.py'>
subprocess   module    <module 'subprocess' from<...>ython3.10/subprocess.py'>
time         module    <module 'time' (built-in)>
tri          module    <module 'matplotlib.tri' <...>plotlib/tri/__init__.py'>

Variable   Type        Data/Info
--------------------------------
spsolve    function    <function spsolve at 0x7f4f38188040>

Variable     Typ

# Mesh Generation

## Sphere model (Thin DL)

# Test Case 1 (Pesudocode)